# Statistical Quality Control Analysis of Service Startup Times

This notebook presents a full simulation and control chart analysis for service startup durations (in milliseconds) using six different quality control techniques:
- x̄-R Chart
- Individual Chart
- Moving Average Chart
- Moving Range Chart
- EWMA Chart
- CUSUM Chart

Sample 10 includes an intentionally introduced assignable cause (outlier).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Veri Üretimi
np.random.seed(42)
num_samples = 25
sample_size = 5
mean_time_ms = 3000
std_dev_ms = 300

data = []
for i in range(num_samples):
    if i == 9:  # assignable cause
        sample = np.random.normal(loc=5000, scale=200, size=sample_size)
    else:
        sample = np.random.normal(loc=mean_time_ms, scale=std_dev_ms, size=sample_size)
    data.append(np.round(sample).astype(int))

df = pd.DataFrame(data, columns=[f"Obs_{i+1}" for i in range(sample_size)])
df["Sample_Mean"] = df.mean(axis=1)
df["Sample_Range"] = df.max(axis=1) - df.min(axis=1)
df["Assignable_Cause"] = ["Yes" if i == 9 else "No" for i in range(num_samples)]

means = df["Sample_Mean"]
ranges = df["Sample_Range"]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Veri Üretimi
np.random.seed(42)
num_samples = 25
sample_size = 5
mean_time_ms = 3000
std_dev_ms = 300

data = []
for i in range(num_samples):
    if i == 9:  # assignable cause
        sample = np.random.normal(loc=5000, scale=200, size=sample_size)
    else:
        sample = np.random.normal(loc=mean_time_ms, scale=std_dev_ms, size=sample_size)
    data.append(np.round(sample).astype(int))

df = pd.DataFrame(data, columns=[f"Obs_{i+1}" for i in range(sample_size)])
df["Sample_Mean"] = df.mean(axis=1)
df["Sample_Range"] = df.max(axis=1) - df.min(axis=1)
df["Assignable_Cause"] = ["Yes" if i == 9 else "No" for i in range(num_samples)]

means = df["Sample_Mean"]
ranges = df["Sample_Range"]

# 2. x̄-R Chart
x_bar_bar = means.mean()
r_bar = ranges.mean()
A2, D3, D4 = 0.577, 0.0, 2.114
ucl_x = x_bar_bar + A2 * r_bar
lcl_x = x_bar_bar - A2 * r_bar
ucl_r = D4 * r_bar
lcl_r = D3 * r_bar

plt.figure(figsize=(12, 6))
plt.plot(means, marker='o', label='Sample Means')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_x, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_x, color='red', linestyle='--', label='LCL')
plt.title('x̄ Chart')
plt.legend(); plt.grid(); plt.show()

plt.figure(figsize=(12, 6))
plt.plot(ranges, marker='s', label='Sample Ranges')
plt.axhline(r_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_r, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_r, color='red', linestyle='--', label='LCL')
plt.title('R Chart')
plt.legend(); plt.grid(); plt.show()

# 3. Individual Chart
overall_std = means.std()
ucl_i = x_bar_bar + 3 * overall_std
lcl_i = x_bar_bar - 3 * overall_std

plt.figure(figsize=(12, 6))
plt.plot(means, marker='o', label='Individual Means')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_i, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_i, color='red', linestyle='--', label='LCL')
plt.title('Individual Chart')
plt.legend(); plt.grid(); plt.show()

# 4. Moving Average Chart
window_size = 3
moving_avg = means.rolling(window=window_size).mean()

plt.figure(figsize=(12, 6))
plt.plot(moving_avg, marker='o', label='Moving Average')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_i, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_i, color='red', linestyle='--', label='LCL')
plt.title('Moving Average Chart')
plt.legend(); plt.grid(); plt.show()

# 5. Moving Range Chart
moving_range = means.diff().abs()
mr_bar = moving_range[1:].mean()
d2 = 1.128
sigma_mr = mr_bar / d2
ucl_mr = 3 * sigma_mr

plt.figure(figsize=(12, 6))
plt.plot(moving_range, marker='o', label='Moving Range')
plt.axhline(mr_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_mr, color='red', linestyle='--', label='UCL')
plt.title('Moving Range Chart')
plt.legend(); plt.grid(); plt.show()

# 6. EWMA Chart
alpha = 0.3
ewma = means.ewm(alpha=alpha, adjust=False).mean()
L = 3
ucl_ewma = x_bar_bar + L * overall_std * np.sqrt(alpha / (2 - alpha))
lcl_ewma = x_bar_bar - L * overall_std * np.sqrt(alpha / (2 - alpha))

plt.figure(figsize=(12, 6))
plt.plot(ewma, marker='o', label='EWMA')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_ewma, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_ewma, color='red', linestyle='--', label='LCL')
plt.title('EWMA Chart')
plt.legend(); plt.grid(); plt.show()

# 7. CUSUM Chart
k = 0.5 * overall_std
h = 5 * overall_std
cusum_pos, cusum_neg = [0], [0]

for i in range(1, len(means)):
    s_pos = max(0, cusum_pos[-1] + (means.iloc[i] - x_bar_bar - k))
    s_neg = min(0, cusum_neg[-1] + (means.iloc[i] - x_bar_bar + k))
    cusum_pos.append(s_pos)
    cusum_neg.append(s_neg)

plt.figure(figsize=(12, 6))
plt.plot(cusum_pos, label='CUSUM Positive', color='blue')
plt.plot(cusum_neg, label='CUSUM Negative', color='orange')
plt.axhline(h, color='red', linestyle='--', label='UCI')
plt.axhline(-h, color='red', linestyle='--', label='LCI')
plt.title('CUSUM Chart')
plt.legend(); plt.grid(); plt.show()
